In [1]:
import pandas as pd
import plotly 
import numpy as np
import plotly.figure_factory as ff
import scipy
import plotly.graph_objects as go
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

#pip install scipy
#!pip install Dash

In [2]:
# Import data

data = pd.read_csv('chocolate.csv')
continent = pd.read_csv('countryContinent.csv',encoding = "ISO-8859-1")
imp_exp=pd.read_csv('UNdata_Export_20220301_151116452.csv')
coord=pd.read_csv('country_points.csv',encoding = "ISO-8859-1")

In [3]:
coord.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [4]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [5]:
imp_exp.head()

,Table Code,Country or Area Code,Country or Area,Year,Comm. Code,Commodity,Flow Code,Flow,Trade (USD),Weight (kg),Quantity Code,Quantity Name,Quantity
0,19,4,Afghanistan,2018,180500,"Cocoa powder, unsweetened",1,Import,7.247329e+04,8.070000e+04,8,Weight in kilograms,8.070000e+04
1,19,4,Afghanistan,2018,180690,Chocolate/cocoa food preparations nes,1,Import,3.281740e+07,5.662823e+06,8,Weight in kilograms,5.662823e+06
2,19,4,Afghanistan,2018,180690,Chocolate/cocoa food preparations nes,2,Export,4.327404e+06,0.000000e+00,8,Weight in kilograms,6.239175e+06
3,19,4,Afghanistan,2018,180690,Chocolate/cocoa food preparations nes,3,Re-Export,7.452492e+04,0.000000e+00,8,Weight in kilograms,0.000000e+00
4,19,4,Afghanistan,2016,180690,Chocolate/cocoa food preparations nes,1,Import,1.568728e+08,2.677301e+07,8,Weight in kilograms,2.677301e+07


In [6]:
imp_exp=imp_exp[imp_exp['Commodity']=='Cocoa beans, whole or broken, raw or roasted'][['Country or Area','Year','Commodity','Flow','Quantity','Trade (USD)']]
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


In [7]:
# Prepare the data for the merge
data["company_location"] = data["company_location"].str.title()
data["country_of_bean_origin"] = data["country_of_bean_origin"].str.title()
data["company_location"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)'}, inplace=True)
data["country_of_bean_origin"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)','Burma':'Myanmar','Tanzania':'Tanzania, United Republic of','Trinidad':'Trinidad and Tobago','Dr Congo':'Congo (Democratic Republic of the)'}, inplace=True)

In [8]:
# removing Unnamed:0
data=data.iloc[:,1:]

In [9]:
teste=data.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'continent':'company_continent','sub_region':'company_region','code_2':'company_code_2'}), left_on='company_location', right_on='country', how='left')

In [10]:
teste[teste['company_continent'].isna()]['company_location'].value_counts

<bound method IndexOpsMixin.value_counts of 193        Wales
486     Scotland
487     Scotland
488     Scotland
489     Scotland
490     Scotland
491     Scotland
1083    Scotland
1084    Scotland
1513    Scotland
1514    Scotland
Name: company_location, dtype: object>

In [11]:
teste=teste[teste['company_location']!= 'Scotland']

In [12]:
teste=teste.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'continent':'bean_continent','sub_region':'bean_region','code_2':'bean_code_2'}), left_on='country_of_bean_origin', right_on='country', how='left')

In [13]:
teste[teste['bean_continent'].isna()]['country_of_bean_origin'].value_counts

<bound method IndexOpsMixin.value_counts of 88      Blend
92      Blend
96      Blend
100     Blend
177     Blend
        ...  
2188    Blend
2201    Blend
2203    Blend
2209    Blend
2213    Blend
Name: country_of_bean_origin, Length: 177, dtype: object>

In [14]:
teste=teste[teste['country_of_bean_origin']!= 'Blend']

In [15]:
data=teste

In [16]:
# Add the coordinates both for bean_origin and company_location
coord.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [17]:
teste1=data.merge(coord.iloc[:,:-1], left_on='bean_code_2', right_on='country', how='left').rename(columns={'latitude':'bean_lat','longitude':'bean_lon'})
teste2=data.merge(coord.iloc[:,:-1], left_on='company_code_2', right_on='country', how='left').rename(columns={'latitude':'country_lat','longitude':'country_lon'})

In [18]:
data1=data.groupby(by=['company']).mean()
data1

,ref,review_date,cocoa_percent,rating,counts_of_ingredients
company,,,,,
5150,2455.333333,2019.000000,76.000000,3.500000,3.000000
A. Morin,1294.480000,2013.960000,68.880000,3.400000,3.960000
AMMA,670.600000,2010.600000,69.000000,3.500000,4.000000
Acalli,1966.000000,2017.500000,71.000000,3.562500,2.000000
Adi aka Fijiana (Easy In Ltd),705.000000,2011.000000,75.000000,3.250000,3.000000
...,...,...,...,...,...
Zotter,1033.705882,2012.647059,70.352941,3.352941,3.823529
hello cocoa,1460.000000,2015.000000,65.500000,3.125000,3.000000
hexx,1848.000000,2016.500000,71.500000,2.925000,2.000000


In [19]:
var1=np.asarray(data1[data1.rating <= data1.rating.quantile(.25)]['cocoa_percent'],dtype=np.float32)
var2=np.asarray(data1[(data1.rating > data1.rating.quantile(.25)) & (data1.rating <= data1.rating.quantile(.50))]['cocoa_percent'],dtype=np.float32)
var3=np.asarray(data1[(data1.rating > data1.rating.quantile(.50)) & (data1.rating <= data1.rating.quantile(.75))]['cocoa_percent'],dtype=np.float32)
var4=np.asarray(data1[data1.rating > data1.rating.quantile(.75)]['cocoa_percent'],dtype=np.float32)

In [20]:
# falta meter titulo xaxis yaxis
labels = ['1st quartile','2nd quartile','3rd quartile','4th quartile']
variables=[var1,var2,var3,var4]
fig = ff.create_distplot(variables,labels, show_hist=False)
fig.show()


### Exports and Imports

In [21]:
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


In [22]:
imp_exp_regions = imp_exp.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'sub_region':'region'}), left_on = 'Country or Area', right_on='country', how='left')

In [23]:
export = imp_exp_regions[imp_exp_regions['Flow'] == 'Export']
export.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2
8,Albania,1997,"Cocoa beans, whole or broken, raw or roasted",Export,3500.0,3447.00,Albania,Europe,Southern Europe,AL
51,Angola,2016,"Cocoa beans, whole or broken, raw or roasted",Export,10.0,25.75,Angola,Africa,Middle Africa,AO
79,Antigua and Barbuda,2000,"Cocoa beans, whole or broken, raw or roasted",Export,127.0,465.00,Antigua and Barbuda,Americas,Caribbean,AG
108,Argentina,1993,"Cocoa beans, whole or broken, raw or roasted",Export,132.0,420.00,Argentina,Americas,South America,AR
110,Armenia,2019,"Cocoa beans, whole or broken, raw or roasted",Export,23806.0,13462.21,Armenia,Asia,Western Asia,AM


In [24]:
export[export['continent'] == 'Europe']['Country or Area'].unique()

array(['Albania', 'Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia',
       'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
       'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania',
       'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Poland',
       'Portugal', 'Romania', 'Russian Federation', 'Serbia', 'Slovakia',
       'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine'],
      dtype=object)

In [25]:
imports = imp_exp_regions[imp_exp_regions['Flow'] == 'Import']
imports.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD),country,continent,region,code_2
0,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0,Albania,Europe,Southern Europe,AL
1,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0,Albania,Europe,Southern Europe,AL
2,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0,Albania,Europe,Southern Europe,AL
3,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0,Albania,Europe,Southern Europe,AL
4,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0,Albania,Europe,Southern Europe,AL


In [26]:
pivot_imports = pd.pivot_table(imports, values = 'Trade (USD)', index = 'continent', columns = 'Year', aggfunc = np.sum)

In [27]:
pivot_imports.index.name = None
pivot_imports

Year,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Africa,NaN,NaN,NaN,572105.0,1944817.0,1259203.0,3.680804e+06,3.146172e+06,7.516860e+06,2.382868e+06,...,2.724588e+07,1.652407e+09,3.124253e+07,8.429142e+07,3.864340e+07,1.710996e+07,1.977022e+07,1.946710e+07,1.238301e+06,3.180144e+03
Americas,NaN,33847189.0,38439802.0,44099672.0,39999341.0,45966597.0,5.615292e+07,7.992683e+07,6.335198e+07,9.428702e+07,...,3.682923e+08,4.040635e+08,3.341691e+08,2.660147e+08,4.241278e+08,3.429008e+08,4.908743e+08,4.319500e+08,4.580726e+08,2.265479e+08
Asia,84981276.0,189457873.0,182257197.0,178398910.0,220703086.0,160839787.0,1.904380e+08,2.746586e+08,2.664382e+08,2.931037e+08,...,2.001048e+09,2.129123e+09,1.814508e+09,1.587756e+09,2.195441e+09,1.741891e+09,1.812279e+09,1.972022e+09,2.074764e+09,1.474099e+08
Europe,59869290.0,119171486.0,124005183.0,504019590.0,927868418.0,834262287.0,1.399687e+09,1.630405e+09,1.978299e+09,1.667038e+09,...,5.236534e+09,6.174367e+09,4.895103e+09,4.551503e+09,5.295596e+09,5.619806e+09,6.454687e+09,5.625416e+09,5.345948e+09,1.869602e+08
Oceania,981118.0,263575.0,179329.0,236528.0,346922.0,329263.0,3.435490e+05,4.244230e+05,3.892580e+05,4.929880e+05,...,6.344571e+06,6.481258e+06,6.484481e+06,5.499694e+06,6.399103e+06,1.216092e+07,9.947090e+06,8.159731e+06,8.891566e+06,NaN


In [28]:
pivot_export = pd.pivot_table(export, values = 'Trade (USD)', index = 'continent', columns = 'Year', aggfunc = np.sum)

In [29]:
pivot_export.index.name = None
pivot_export.index

Index(['Africa', 'Americas', 'Asia', 'Europe', 'Oceania'], dtype='object')

In [30]:
#frame1 = []
#frame2 = []

frames = []

for year in pivot_export.columns[1:]: # we do not include the first frame here
    frames.append(dict(data=dict(type='bar',
                                 x=pivot_export.index,
                                 y=pivot_export[year],
                                 name=year,
                                ),
                       layout=go.Layout(title_text='Exports in the year of ' + str(year))
                      )
                )

#for year in pivot_imports.columns[1:]: # we do not include the first frame here
#    frames.append(dict(data=dict(type='scatter',
#                                 x=pivot_imports.index,
#                                 y=pivot_imports[year],
#                                 name=year,
#                                ),
#                       layout=go.Layout(title_text='Imports in the year of ' + str(year))
#                      )
#                )

    
#frames = [frame1, frame2]

# First introduction to buttons
fig_bar_layout = dict(title=dict(text='Exports by continent from 1988 to 2019'),
                      yaxis=dict(title='Exports',
                                 range=[0,9000000000]
                                ),
                      updatemenus=[dict(type="buttons",
                                    buttons=[dict(label="Play",
                                                  method="animate",
                                                  args=[None])
                                            ]
                                       )
                                  ]
                    )

initial_data = dict(type='bar', 
                    x=pivot_export.index, 
                    y=pivot_export.values[0],
                    name=str(1988))

fig_bar = go.Figure(data = initial_data, layout = fig_bar_layout, frames=frames)

fig_bar.show(renderer='iframe')

In [31]:
# try dash

In [32]:
data.columns

Index(['ref', 'company', 'company_location', 'review_date',
       'country_of_bean_origin', 'specific_bean_origin_or_bar_name',
       'cocoa_percent', 'rating', 'counts_of_ingredients', 'beans',
       'cocoa_butter', 'vanilla', 'lecithin', 'salt', 'sugar',
       'sweetener_without_sugar', 'first_taste', 'second_taste', 'third_taste',
       'fourth_taste', 'country_x', 'company_continent', 'company_region',
       'company_code_2', 'country_y', 'bean_continent', 'bean_region',
       'bean_code_2'],
      dtype='object')

In [33]:
yes_ingredients = [{'label': 'Vanilla', 'value': 'vanilla'},
           {'label': 'Lecithin', 'value': 'lecithin'},
           {'label': 'Salt', 'value': 'salt'},
           {'label': 'Sweetener without sugar', 'value': 'sweetener_without_sugar'},
           {'label': 'Sugar', 'value': 'sugar'}]

In [34]:
ing_checkbox = dcc.Checklist(
        id='ing_checkbox',
        options=yes_ingredients,
        value=['sugar']
    ),

In [73]:
# The app itself

app = dash.Dash(__name__)

app.layout = html.Div([
    
    html.Div(
            [ html.Div(
                    [   html.Img(
                            src=app.get_asset_url('logo.png'),
                            alt="IMS logo",
                            id="plotly-image",
                            style={
                                "height": "80px",
                                "width": "auto",
                                "margin-bottom": "50px",
                            },
                        )
                    ],
                    className="upper",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.H2(
                                    "Chocolate overview",
                                    style={"font-weight": "bold"},
                                ),
                                html.H3(
                                    "bla bla bla \n bla bla bla", style={"margin-top": "0px"}
                                ),
                            ]
                        )
                    ],
                    className="upper",
                    id="title",
                ),
                html.Div(
                    # create empty div for align cente
                    className="upper",
                ),
            ],
            id="header",
            className="row flex-display",
            style={"margin-bottom": "20px"},
        ),
    



    html.H6('Choose your chocolate'),

    html.Label('The ingredients'),
        
        html.Div([
            html.Div([
                    html.Label("Choose the ingredients:"),
                    html.Br(),
                    html.Br(),
                    dcc.Checklist(
                                id='ing_checkbox',
                                options=yes_ingredients,
                                value=['sugar']
                            ),
            ], className='box', style={'margin': '10px', 'padding-top':'15px', 'padding-bottom':'15px'}),

            html.Div([
                html.Div([
                    html.Label('Results', style={'font-size': 'medium'}),
                    html.Br(),
                    html.Br(),
                    html.Div([
                        
                        html.Div([
                            html.H4('Company', style={'font-weight':'normal'}),
                            html.H3(id="name_company_id")
                        ],className='box_ing'),
                        
                        html.Div([
                            html.H4('Rating', style={'font-weight':'normal'}),
                            html.H3(id="rating_id")]),
                        ],className='box_ing'),

                        html.Div([
                            html.H4('Country', style={'font-weight':'normal'}),
                             html.H3(id="country_id"),
                        ],className='box_ing'),
                    
                       
                        
                        ], style={'display': 'flex'}),

                    ], className='box', style={'heigth':'10%'}),
 
                html.Div([ 
                    html.Div([
                        
                        html.Div([
                            html.Br(),
                            html.Label(id='title_map', style={'font-size':'medium'}), 
                            html.Br(),
                            html.Label('BLA BLA BLA', style={'font-size':'9px'}),
                        ], style={'width': '70%'}),
                        html.Div([

                        ], style={'width': '5%'}),
                       
                    
                    dcc.Graph(id='graph1'),

                ], className='box', style={'padding-bottom': '0px'}), 
                    ]),
                ], style={'width': '60%'}),           
            ], className='row')
            
          

In [74]:
@app.callback(
   
 [
    Output("name_company_id", "children"),
    Output("rating_id", "children"),
    Output("country_id", "children"),
    Output('graph1', 'figure')
    ],
 
    [Input('ing_checkbox', 'value')]
)

def update_graph(ing_chosen):
    #filtered_by_year_df = data[(data['year'] >= year[0]) & (data['year'] <= year[1])]
    
    filtered=data

    for ing in ing_chosen:
        
        criteria = filtered[ing].map(lambda x: x.startswith('have_not'))
        filtered=filtered[-criteria]     
        
    maxi=np.max(filtered['rating'])
    filtered=filtered[filtered['rating']==maxi]
    data_inter = dict(type='bar', x=filtered['company'], y=filtered['rating'])
   
    layout_1 = dict(xaxis=dict(title='Companies'),
                   yaxis=dict(title='Ratings'))
    
    fig = go.Figure(data=data_inter, layout=layout_1)  

    name_company=str(filtered.head(1)['company'].values[0])
    rating=str(filtered.head(1)['rating'].values[0])
    country=str(filtered.head(1)['company_location'].values[0])
    
    
    return name_company,  \
           rating , \
           country , \
           fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
#data[data['rating']==5].head(1)['rating'].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0